## 1. ヒストリカルデータの取得・収益率等の計算

#### 問１：データ取得
##### (1-1) Yahoo Financeを用いて,2005年1月1日から現在までのFord社（'F'）とマクロソフト (‘MSFT’) ，アップル（'AAPL'）の株価を取得しましょう（FordのtickerはFです）

In [2]:
# ライブラリが存在しない場合は，pip install でインストールして下さい
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


tickers = ['F', 'MSFT', 'AAPL'] # 取得するティッカー
data = pd.DataFrame()

for ticker in tickers:
    data[ticker] = yf.download(ticker, start='2005-1-1')['Adj Close']

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


##### (1-2)  .**head()** , **.tail()** メソッドを使ってデータの中身を確認しましょう。

In [3]:
data.head()

,F,MSFT,AAPL
Date,,,
2005-01-03,8.211229,18.732996,0.959292
2005-01-04,8.183323,18.803051,0.969144
2005-01-05,8.054933,18.761026,0.977632
2005-01-06,8.066101,18.740002,0.978389
2005-01-07,8.177740,18.683952,1.049628


#### 問２：収益率・ボラティリティの計算
##### (2-1) 上記の期間における 各銘柄の算術収益率・対数収益率を計算しましょう．

$$
\frac{P_1 - P_0}{P_0} = \frac{P_1}{P_0} - 1
$$

In [4]:
# 算術収益率
simple_return = (data / data.shift(1)) - 1
# 対数収益率
log_return = np.log(data / data.shift(1))

##### (2-2) 各銘柄リターンのボラティリティ（日次・年次）を計算しましょう．

In [5]:
# 算術収益率
daily_simple_vols = simple_return.std()
yearly_simple_vols = simple_return.std() * 250 ** 0.5
# 対数収益率
daily_log_vols = log_return.std()
yearly_log_vols = log_return.std() * 250 ** 0.5

#### 問３：システマティックリスク・アンシステマティック
##### (3-1) ポートフォリオのウェイトを適当に決め，ポートフォリオの分散を計算してみましょう．

In [6]:
# ウェイト
weights = np.array([0.5, 0.2, 0.3])

# ポートフォリオの分散を計算
pfolio_var = np.dot(weights.T, np.dot(simple_return.cov() * 250, weights)) # 年率換算
pfolio_var

0.08631428744871761

##### (3-2) システマティックリスク・アンシステマティックを計算してみましょう．
| システマティックリスク | アンシステマティック |
| ------------------ | ------------------ |
| 日々の株価の変化のことを意味しており，それは全ての会社に影響を与える事象によってもたらされる． | 企業特有，もしくは産業特有のリスクであり，分散投資を通じてリスクを減らすことができる． |

In [14]:
# 各銘柄の分散を計算
vars = np.zeros(3)
for i, ticker in enumerate(tickers):
                                                  vars[i] = simple_return[ticker].var() * 250

# 分散可能なリスクの計算
unsistematic_risk = pfolio_var
for i, var in enumerate(vars):
                                                  unsistematic_risk = unsistematic_risk - (weights[i] ** 2 *var)
# 分散不可能なリスク
sistematic_risk = pfolio_var - unsistematic_risk

#### 問４
##### これまで実施した分析を行うための関数を作成してみましょう．引数は「ティッカー，開始日，採用する収益率」としてみましょう．